In [13]:
import datetime
import pandas as pd
import os
import json
import numpy as np

# Get the current date and time
current_datetime = datetime.datetime.now()
Compliance_report = {'Time of this report generated': str(current_datetime),
                    'Minimum # of long stocks': None,
                    'Minimum # of short stocks': None,
                    'Position Weight': None,
                    'Dollar Neutrality': None,
                    'Cash position': None,
                    'Reminder 1': 'please limit the transactions per week under 35',
                    'Reminder 2': 'Grey Area positions are listed in Grey_area_positions.csv'}
position_weight_threshold = 0.04
long_short_ratio_allowance = 0.05
File_name = input('please input the file names (ex. OpenPosition_11_29_2023.csv)') #example input: OpenPosition_11_29_2023.csv
Cash_value = float(input('please input the cash balance(0): ').replace(',', ''))#example input: 0
Portfolio_value = float(input('please input the portfolio value(985,191.77): ').replace(',', ''))#example input: 980,523.08
df_positions = pd.read_csv(File_name)
df_positions.loc[df_positions['Quantity']>0,'Long(1)/Short(-1)'] = 'Long'
df_positions.loc[df_positions['Quantity']<0,'Long(1)/Short(-1)'] = 'Short'
num_long_stocks = df_positions.groupby('Long(1)/Short(-1)').count()['Symbol']['Long']
if num_long_stocks < 50:
    Compliance_report['Minimum # of long stocks'] = "Violated! " +str(50-num_long_stocks)+ " more long stocks should be added to position."
else:
    Compliance_report['Minimum # of long stocks'] = 'Complied.'
num_short_stocks = df_positions.groupby('Long(1)/Short(-1)').count()['Symbol']['Short']
if num_short_stocks < 50:
    Compliance_report['Minimum # of short stocks'] = "Violated! " +str(50-num_short_stocks)+ " more short stocks should be added to position."
else:
    Compliance_report['Minimum # of short stocks'] = 'Complied.'
df_positions['Position_Weight'] = df_positions['MarketValue']/Portfolio_value
Violations_position_weight = df_positions.loc[df_positions['Position_Weight'] > position_weight_threshold]

if Violations_position_weight.shape[0] > 0:
    Compliance_report['Position Weight'] = 'Violated! '+ str(Violations_position_weight.shape[0]) +' violations: the violations has been exported to a csv file in the Compliance_report folder(Violations_Position_Weight.csv)'
    if not os.path.exists('Compliance_report'):
        os.makedirs('Compliance_report')
    Violations_position_weight.to_csv('Compliance_report/0. Violations_Position_Weight.csv')
else:
    Compliance_report['Position Weight'] = 'Complied.'
Long_market_value = df_positions.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Long']
Short_market_value = df_positions.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Short']
if (Long_market_value/Short_market_value)<(1+long_short_ratio_allowance) and (Long_market_value/Short_market_value)> (1-long_short_ratio_allowance):
    Compliance_report['Dollar Neutrality'] = "Complied. the long/short ratio is " + str(Long_market_value/Short_market_value)
else:
    Compliance_report['Dollar Neutrality'] = "Violated! the long/short ratio is " + str(Long_market_value/Short_market_value)
if Cash_value/Portfolio_value > position_weight_threshold:
    Compliance_report['Cash position'] = "Violated! the cash value now is "+str(Cash_value)+ "("+str(100*Cash_value/Portfolio_value)+"%); Target: < "+str(Portfolio_value*position_weight_threshold)+"; please buy/short some new stocks or increase the positions of some current stocks. Some recommendations is been provided in the Compliance_report folder. (Low_weight_long_positions.csv)"
else:
    Compliance_report['Cash position'] = "Complied."
if not os.path.exists('Compliance_report'):
        os.makedirs('Compliance_report')
temp_filter = (df_positions['Long(1)/Short(-1)'] == "Long")&(df_positions['Position_Weight'] < 0.0075)
df_positions[temp_filter].sort_values(by='Position_Weight').to_csv('Compliance_report/3. Low_weight_long_positions.csv')
temp_filter = (df_positions['Long(1)/Short(-1)'] == "Short")&(df_positions['Position_Weight'] < 0.0075)
df_positions[temp_filter].sort_values(by='Position_Weight').to_csv('Compliance_report/4. Low_weight_Short_positions.csv')
df_positions.loc[(df_positions['Position_Weight'] > position_weight_threshold * 0.75)&((df_positions['Position_Weight'] < position_weight_threshold))].to_csv('Compliance_report/2. Grey_Area_positions.csv')

with open('Compliance_report/1. Compliance_Summary.txt', 'w') as file:
    json.dump(Compliance_report, file, indent=4)
print('Finished checking! Please look at the compliance report in the [Compliance_report] folder')

please input the file names (ex. OpenPosition_11_29_2023.csv)OpenPosition_11_29_2023.csv
please input the cash balance(0): 0
please input the portfolio value(985,191.77): 985,191.77
Finished checking! Please look at the compliance report in the [Compliance_report] folder


/var/folders/2j/xqk1xvts58v88hvpqrdt8thw0000gn/T/ipykernel_31738/3670627159.py:45: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Long_market_value = df_positions.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Long']
/var/folders/2j/xqk1xvts58v88hvpqrdt8thw0000gn/T/ipykernel_31738/3670627159.py:46: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Short_market_value = df_positions.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Short']


# Emergency action

In [14]:
weight_renew_standard = 0.02
def get_simulation_cash_weight(cash_value_, portfolio_value_):
    return cash_value_/portfolio_value_
def get_simulation_long_short_ratio(long_value_, short_value_):
    return long_value_/short_value_
df_simulator = df_positions.copy()
simulation_cash_values = Cash_value
simulation_Portfolio_value = Portfolio_value 

# Long short ratio
simulation_Long_market_value = df_simulator.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Long']
simulation_Short_market_value = df_simulator.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Short']
simulation_long_short_ratio = get_simulation_long_short_ratio(simulation_Long_market_value, simulation_Short_market_value)

# transaction recorder
simulator_transaction_recorder = pd.DataFrame(columns=['Symbol', 'Buy/Sell/Short/Cover', 'Quantity', 'Price_assumption', 'Market_value_assumptions'])
#pd.Series({'Symbol': 5, 'Buy/Sell/Short/Cover': 'Buy', 'Quantity': 10, 'Price_assumption': 10, 'Market_value_assumptions':10})

def update_df_simulator(ticker, amount):
    global simulation_cash_values
    global simulation_long_short_ratio
    global simulation_Portfolio_value
    if df_simulator.loc[df_simulator['Symbol'] == ticker, 'Quantity'].values[0] * amount > 0:
        temp_directiont = 1
    else:
        temp_directiont = -1
    
    df_simulator.loc[df_simulator['Symbol'] == ticker, 'Quantity'] += amount
    delta_market_value_of_position = temp_directiont*abs(amount*(df_simulator.loc[df_simulator['Symbol'] == ticker, 'LastPrice']))
    df_simulator.loc[df_simulator['Symbol'] == ticker, 'MarketValue'] += delta_market_value_of_position
    simulation_Long_market_value = df_simulator.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Long']
    simulation_Short_market_value = df_simulator.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Short']
    simulation_long_short_ratio = get_simulation_long_short_ratio(simulation_Long_market_value, simulation_Short_market_value)
    #update cash
    simulation_cash_values +=((-1)*delta_market_value_of_position.values[0] )
    df_simulator['Position_Weight'] = df_simulator['MarketValue']/simulation_Portfolio_value

    
df_simulator['Position_Weight'] = df_simulator['MarketValue']/simulation_Portfolio_value
simulator_Violations_position_weight = df_simulator.loc[df_simulator['Position_Weight'] > position_weight_threshold]

# indentify violations
if simulator_Violations_position_weight.loc[simulator_Violations_position_weight['Quantity'] > 0].shape[0] != 0:
    simulator_Violations_position_weight = df_simulator.loc[df_simulator['Position_Weight'] > position_weight_threshold]
    simulator_Violations_position_weight.loc[simulator_Violations_position_weight['Quantity'] > 0, 'Rectifying_direction'] = -1
    simulator_Violations_position_weight.loc[simulator_Violations_position_weight['Quantity'] < 0, 'Rectifying_direction'] = 1
    # Calculating action
    simulator_Violations_position_weight.loc[:,'Recommended_action'] = (((simulator_Violations_position_weight['Position_Weight'] - weight_renew_standard)*simulator_Violations_position_weight['Rectifying_direction']*simulation_Portfolio_value)/simulator_Violations_position_weight['LastPrice']).round(0) 
    simulator_Violations_position_weight.loc[:, 'Recommended_effect'] = simulator_Violations_position_weight['Recommended_action']*simulator_Violations_position_weight['LastPrice']
    simulator_Violations_position_weight.loc[simulator_Violations_position_weight['Quantity'] > 0, 'Rectifying_direction'] = 'Sell'
    simulator_Violations_position_weight.loc[simulator_Violations_position_weight['Quantity'] < 0, 'Rectifying_direction'] = 'Cover'
    # recording action
    temp_recorder = simulator_Violations_position_weight.loc[:,['Symbol', 'Rectifying_direction', 'Recommended_action', 'LastPrice', 'Recommended_effect']].copy()
    temp_recorder.columns = ['Symbol', 'Buy/Sell/Short/Cover', 'Quantity', 'Price_assumption', 'Market_value_assumptions']
    simulator_transaction_recorder = pd.concat((simulator_transaction_recorder, temp_recorder))
    ## Update Simulator
    for i in range(simulator_Violations_position_weight.shape[0]):
        simulator_Violations_position_weight.iloc[i,:]
        _ticker = simulator_Violations_position_weight.iloc[i,:]['Symbol']
        _amount = simulator_Violations_position_weight.iloc[i,:]['Recommended_action']
        update_df_simulator(ticker = _ticker, amount = _amount)

    
short_flag = True
short_pass = False
Long_flag = True
Long_pass = False
while (simulation_cash_values/simulation_Portfolio_value) >  position_weight_threshold:
    #print('while in')
    if df_simulator[df_simulator['Position_Weight'] < weight_renew_standard - 0.001].shape[0] <= 0:
        print('There is still overweighted position in Cash, Please buy more or short more stocks manually')
        break
    
    
    if (simulation_long_short_ratio > (1 ) and short_flag) or short_pass:
        #print('short processing')
        temp_long_short_rebalancer = df_simulator.loc[df_simulator['Quantity'] < 0, :].copy()
        if temp_long_short_rebalancer.shape[0] == 0:
            short_flag = False
            Long_pass = True
            #print('switch off _ short')
            continue
        temp_long_short_rebalancer = temp_long_short_rebalancer.loc[temp_long_short_rebalancer['Position_Weight'] < weight_renew_standard, :]
        temp_long_short_rebalancer['Rectifying_direction'] = -1
        temp_long_short_rebalancer['Recommended_action'] = (-1) *((weight_renew_standard - temp_long_short_rebalancer['Position_Weight'])*simulation_Portfolio_value/temp_long_short_rebalancer['LastPrice'])
        temp_long_short_rebalancer['Recommended_action'] = np.floor(temp_long_short_rebalancer['Recommended_action'])
        temp_long_short_rebalancer['Recommended_effect'] = temp_long_short_rebalancer['Recommended_action']*temp_long_short_rebalancer['LastPrice']
        temp_long_short_rebalancer['Rectifying_direction'] = 'Short'
        if temp_long_short_rebalancer.shape[0] == 0:
            short_flag = False
            Long_pass = True
            #print('switch off _ short')
            continue
        #print('short processing 2')
        
    #Temp_flag = True
    
    
    if (simulation_long_short_ratio < (1) and Long_flag) or Long_pass :
        #print('Long processing')
        temp_long_short_rebalancer = df_simulator.loc[df_simulator['Quantity'] > 0, :].copy()
        if temp_long_short_rebalancer.shape[0] == 0:
            Long_flag = False
            short_pass = True
            #print('switch off _ long')
            #Temp_flag = False
            continue
            
        temp_long_short_rebalancer = temp_long_short_rebalancer.loc[temp_long_short_rebalancer['Position_Weight'] < weight_renew_standard, :]
        temp_long_short_rebalancer['Rectifying_direction'] = 1
        temp_long_short_rebalancer['Recommended_action'] = (1) *((weight_renew_standard - temp_long_short_rebalancer['Position_Weight'])*simulation_Portfolio_value/temp_long_short_rebalancer['LastPrice'])
        temp_long_short_rebalancer['Recommended_action'] = np.ceil(temp_long_short_rebalancer['Recommended_action'])
        temp_long_short_rebalancer['Recommended_effect'] = temp_long_short_rebalancer['Recommended_action']*temp_long_short_rebalancer['LastPrice']
        temp_long_short_rebalancer['Rectifying_direction'] = 'Buy'
        
        if temp_long_short_rebalancer.shape[0] == 0:
            Long_flag = False
            short_pass = True
            #print('switch off _ long')
            #Temp_flag = False
            continue
        
        #print('Long processing 2')

    
    # recording action
    # print('begin recording')
    temp_long_short_rebalancer = temp_long_short_rebalancer.sort_values(by = 'Position_Weight').iloc[0,:][['Symbol', 'Rectifying_direction', 'Recommended_action', 'LastPrice', 'Recommended_effect']]
    temp_long_short_rebalancer = temp_long_short_rebalancer.to_frame().T
    temp_recorder = temp_long_short_rebalancer
    temp_recorder.columns = ['Symbol', 'Buy/Sell/Short/Cover', 'Quantity', 'Price_assumption', 'Market_value_assumptions']
    simulator_transaction_recorder = pd.concat((simulator_transaction_recorder, temp_recorder))
    ## Update Simulator
    _ticker = temp_long_short_rebalancer.iloc[0,:]['Symbol']
    _amount = temp_long_short_rebalancer.iloc[0,:]['Quantity']
    update_df_simulator(ticker = _ticker, amount = _amount)
    #print('success')
    

if not os.path.exists('Emergency_Action'):
        os.makedirs('Emergency_Action')
simulator_transaction_recorder.to_csv('Emergency_Action/Emergency_Action.csv')
df_simulator.to_csv('Emergency_Action/Simulated_Position_After_Emergency_Action.csv')

print('Finished Action Generating, Please check the file in [Emergency_Action/Emergency_Action.csv]')
print('Please check the simulated position in [Emergency_Action/Simulated_Position_After_Emergency_Action.csv]')
print('The updated long/short ratio after emergency action will be ', simulation_long_short_ratio)
print('The updated cash after emergency action will be ', simulation_cash_values)
print('The updated Cash weight after emergency action will be ', simulation_cash_values/simulation_Portfolio_value)


from datetime import timedelta
from datetime import date
from datetime import datetime
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np

current_datetime = datetime.now()

Event_day = current_datetime#datetime(2022, 1, 28)
Days_for_abonormal_returns = 0 # +-20
Days_before_to_construct_model = 730 #300 2000 -> find out alpha do not have persistence

Days_for_abonormal_returns = timedelta(days=Days_for_abonormal_returns)
Days_before_to_construct_model = -timedelta(days=Days_before_to_construct_model)

Days_range_for_contstruct_model = (Days_before_to_construct_model, -Days_for_abonormal_returns)
Days_range_for_contstruct_model = (Event_day+Days_range_for_contstruct_model[0], 
                               Event_day+Days_range_for_contstruct_model[1])



print('Analysing Portfolio Risk Exposures............Please wait for the result ')

START_DATE = Days_range_for_contstruct_model[0].strftime('%Y-%m-%d')
# three factors
df_three_factor = web.DataReader('F-F_Research_Data_Factors_daily',
                                                'famafrench', start=START_DATE)[0]
df_three_factor.index = df_three_factor.index.format()
# momentum factor
df_mom = web.DataReader('F-F_Momentum_Factor_daily', 'famafrench',
                                       start=START_DATE)[0]
df_mom.index = df_mom.index.format()
# five factors
df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily',
                                               'famafrench',
                                               start=START_DATE)[0]
df_five_factor.index = df_five_factor.index.format()


END_DATE = df_five_factor.index[-1] # not equal today due to website update  speed

def FF5_fit(RISKY_ASSET = 'C', START_DATE = '2013-12-31', END_DATE = '2018-12-31'):

    global df_five_factor
    
    #downloading asset data
    asset_df = yf.download(RISKY_ASSET,
                                      start=START_DATE,
                                      end=END_DATE,
                                      progress=False)

    y = asset_df['Adj Close'].resample('D').last().pct_change().dropna()
    y.index = y.index.strftime('%Y-%m-%d')
    y.name = 'return'


    # join all datasets on the index
    five_factor_data = df_five_factor.join(y)
    # rename columns
    five_factor_data.columns = ['mkt', 'smb', 'hml', 'rmw', 'cma',
    'rf', 'rtn']
    # divide everything (except returns) by 100
    five_factor_data.loc[:, five_factor_data.columns != 'rtn'] /= 100


    # Change the index type
    five_factor_data.index = five_factor_data.index.map(str)
    # convert index to datetime
    five_factor_data.index = pd.to_datetime(five_factor_data.index,
                                                 format='%Y-%m-%d')
    # select period of interest
    five_factor_data = five_factor_data.loc[START_DATE:END_DATE]
    # calculate excess returns
    five_factor_data['excess_rtn'] = five_factor_data.rtn - five_factor_data.rf


    five_factor_model = smf.ols(
             formula='excess_rtn ~ mkt + smb + hml + rmw + cma',
             data=five_factor_data
         ).fit()
    
    return five_factor_model, five_factor_data, asset_df
# a,b,c = FF5_fit(RISKY_ASSET = 'C', START_DATE = START_DATE, END_DATE = END_DATE)
# a.params


Betas = []
residuals = []
r_squareds = []
for i in df_positions['Symbol'].tolist():
    ff_model, b, c = FF5_fit(RISKY_ASSET = i, START_DATE = START_DATE, END_DATE = END_DATE)
    residuals.append(ff_model.resid)
    r_squareds.append(ff_model.rsquared)
    ff_model = ff_model.params.to_frame().T
    ff_model['Ticker'] = i
    Betas.append(ff_model)
    
    
Betas = pd.concat(Betas)
df_positions = pd.merge(df_positions, Betas, left_on='Symbol', right_on='Ticker', how='inner')
df_positions['Market_Position'] = df_positions['Quantity']*df_positions['LastPrice']
df_positions['Beta_weight'] = df_positions['Market_Position']/df_positions['MarketValue'].sum()
Portfolio_betas = {}
for i in Betas.columns[1:-1]:
    Portfolio_betas[i] = (df_positions[i]*df_positions['Beta_weight']).sum()*2

with open('Compliance_report/5. Exposure_to_Systematic_Risk.txt', 'w') as file:
    json.dump(Portfolio_betas, file, indent=4)

    
print('Finished Analysing Portfolio Risk Exposures. Please check [Emergency_Action/5. Exposure_to_Systematic_Risk.txt]')

/var/folders/2j/xqk1xvts58v88hvpqrdt8thw0000gn/T/ipykernel_31738/393550857.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  simulation_Long_market_value = df_simulator.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Long']
/var/folders/2j/xqk1xvts58v88hvpqrdt8thw0000gn/T/ipykernel_31738/393550857.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  simulation_Short_market_value = df_simulator.groupby('Long(1)/Short(-1)').sum()['MarketValue']['Short']


Finished Action Generating, Please check the file in [Emergency_Action/Emergency_Action.csv]
Please check the simulated position in [Emergency_Action/Simulated_Position_After_Emergency_Action.csv]
The updated long/short ratio after emergency action will be  1.0121890754435072
The updated cash after emergency action will be  0.0
The updated Cash weight after emergency action will be  0.0
Analysing Portfolio Risk Exposures............Please wait for the result 
Finished Analysing Portfolio Risk Exposures. Please check [Emergency_Action/5. Exposure_to_Systematic_Risk.txt]


In [15]:
df_residuals = pd.concat(residuals, axis = 1)
df_residuals.columns = df_positions['Symbol']

In [16]:
np.array(r_squareds).mean()

0.3870830297317935

In [17]:
df_positions.to_csv('Portfolio_Betas.csv')

In [18]:
df_residuals.to_csv('Portfolio_Residuals.csv')


MarketValue = abs(LastPrice x Quantity)
Market_Position = LastPrice x Quantity
Beta_weight = Market_Position / sum(MarketValue)


porfolio beta (i) = sum(Beta_asset_i * Beta_weight)*2
